# Peak analysis from MACS2 output

This notebook analyzes peaks from MACS2 output.

1. Calculate average peak position from replicates

2. Identify nearest ORF adjacent to each peak and plot distribution of distance to the nearest ORF

3. Cross-reference peaks with RNA-seq table and assign "activate" or "repressed" type to each peak

In [4]:
import os
import sys


SCRIPTS = "/Users/yunfei/GeisingerLab/2023ChipSeq/src"
print(sys.version)


3.11.3 (main, May 15 2023, 10:43:03) [Clang 14.0.6 ]


## Calculate average peak position from replicates

### Use bedtools multiinter to identify intersected peaks

In [33]:
INDIR = "/Users/yunfei/2023_ChipSeq/MACS2_output_500k_ext200_input28-2/narrowPeak"
OUTDIR = "/Users/yunfei/2023_ChipSeq/bed_multiinter"

!mkdir -p $OUTDIR

import re
for seed in [1, 4, 7]:
    for sample in [28, 49]:
        replicates = []
        for file in os.listdir(INDIR):
            if sample == 28: # for 28, only include replicate 2, and populate the list with 3 x replicate 2
                if re.match("^.+" + str(sample) + "-2.seed" + str(seed) + ".+$", file):
                    replicates.append(os.path.join(INDIR, file))
            else:
                if re.match("^.+" + str(sample) + "-\d.seed" + str(seed) + ".+$", file):
                    replicates.append(os.path.join(INDIR, file))
        if len(replicates) == 1:
            replicates += replicates
        outfile =os.path.join(OUTDIR, ("BfmR-ChIP-" + str(sample) + "_seed" + str(seed) + ".intersect.bed"))
        cmd = "bedtools multiinter -cluster -header -i " + " ".join(replicates) + " > " + outfile # 23Feb2023: added -header option so the output contains file path
        os.system(cmd)

### Calculate average peak position
